In [ ]:
import pandas as pd
import numpy as np
import re
import pathlib
import functools

In [ ]:
df = pd.read_csv("../data/daily/AAPL.csv") 

In [ ]:
df.columns

In [ ]:
def get_real_name(x):
  
  if '.' in x:
    return x.split('.')[1]
  else:
    return x


In [ ]:
get_real_name('AAPL.Close')

In [ ]:
new_columns = [get_real_name(name) for name in df.columns.to_list()]
new_columns

In [ ]:
df.columns = new_columns

In [ ]:
df

In [ ]:
df[['Open', 'High', 'Low', 'Close', 'Volume']] = np.log(df[['Open', 'High', 'Low', 'Close', 'Volume']])

In [ ]:
df['Mean'] = (df['Close'] + df['Open'])/2
df['Mean_diff'] = df['Mean'].diff()

In [ ]:
n=20
alpha = 2/(n+1)
alpha

In [ ]:

df['Mean_diff_EMA'] = df['Mean_diff'].ewm(alpha=alpha).mean()

In [ ]:
df['Mean_diff_EMS'] = df['Mean_diff'].ewm(alpha=alpha).std()

In [ ]:
df

In [ ]:
df['Mean_diff_next'] = df['Mean_diff'].shift(-1)

In [ ]:
df

In [ ]:
df[['Mean_diff_EMA', 'Mean_diff_EMS', 'Mean_diff_next']].corr()

In [ ]:
df['Mean_diff_EMZ'] = df['Mean_diff_EMA'] / df['Mean_diff_EMS']

In [ ]:
df[['Mean_diff_EMA', 'Mean_diff_EMS', 'Mean_diff_EMZ','Mean_diff_next']].corr()

In [ ]:
def get_df(name):
  df = pd.read_csv(name)
  new_columns = [get_real_name(name) for name in df.columns.to_list()]
  df.columns = new_columns
  df[['Open', 'High', 'Low', 'Close', 'Volume']] = np.log(df[['Open', 'High', 'Low', 'Close', 'Volume']])
  df['Mean'] = (df['Close'] + df['Open'])/2
  df['Mean_diff'] = df['Mean'].diff()
  df['Close_diff'] = df['Close'].diff()
  df['Close_diff_next'] = df['Close_diff'].shift(-1)
  n=20
  alpha = 2/(n+1)
  alpha
  
  df['Mean_diff_EMA'] = df['Mean_diff'].ewm(alpha=alpha).mean()
  df['Mean_diff_EMS'] = df['Mean_diff'].ewm(alpha=alpha).std()
  df['Mean_diff_EMZ'] = df['Mean_diff_EMA'] / df['Mean_diff_EMS']
  return df

In [ ]:
df = get_df("../data/daily/A.csv")

In [ ]:
df[['Mean_diff_EMA', 'Mean_diff_EMS', 'Mean_diff_EMZ','Close_diff_next']].corr()

In [ ]:
def concat(df1, df2):
  df = pd.concat([df1, df2])
  return df

In [ ]:
dir = "../data/daily/"
p = pathlib.Path(dir).glob('*.csv')
files = [x for x in p if x.is_file()]
df = None
for file in files:
  df1 = get_df(file)
  df = df1 if df is None else pd.concat([df, df1])
df


In [ ]:
df[['Mean_diff_EMA', 'Mean_diff_EMS', 'Mean_diff_EMZ','Close_diff_next']].corr()

In [ ]:
df[(df['Mean_diff_EMZ'] > -3) & (df['Mean_diff_EMZ'] < 3)]['Mean_diff_EMZ'].hist(bins=100)

In [ ]:
df[df['Mean_diff_EMZ']<-1][['Mean_diff_EMA', 'Mean_diff_EMS', 'Mean_diff_EMZ','Close_diff_next']].corr()

In [ ]:
df

In [ ]:
import plotly
import plotly.express as px

In [ ]:
df_p = df[(df['Mean_diff_EMZ'] > -3) & (df['Mean_diff_EMZ'] < 3)]
fig = px.scatter(x=df_p['Mean_diff_EMZ'], y=df_p['Close_diff_next'])
fig.show()

In [ ]:
df[df['Mean_diff_EMZ']>-3][['Mean_diff_EMZ', 'Close_diff_next']].reset_index(drop=True).plot()

In [ ]:
df[df['Mean_diff_EMZ']<-1.2]

In [ ]:
# calculate the rank:
# merge the table with its date
def merge_2_df(df1, df2):
  return df1.merge(df2, on='Date', how='outer')

In [ ]:
dir = "../data/daily/"
p = pathlib.Path(dir).glob('*.csv')
files = [x for x in p if x.is_file()][:10]
df = None
for file in files:
  stock_name = str(file).split('/')[-1].split('.')[0]
  df1 = get_df(file)
  df1.columns = [stock_name + '.' + x if x != 'Date' else x for x in df1.columns]
  df = df1 if df is None else df.merge(df1, on='Date', how='outer')
df

In [ ]:
emz_columns = df.columns[['EMZ' in x for x in df.columns]]

In [ ]:
emz_columns

In [ ]:
df[emz_columns]

In [ ]:
df[emz_columns].apply(lambda x: , axis='columns')

In [ ]:
s = [2, 3, 1, 4, 5, 3]
s1 = sorted(range(len(s)), key=lambda k: s[k])
s1

In [ ]:
s2 = [0] * 6
for idx, a in enumerate(s1):
  s2[a] = idx


In [ ]:
s2

In [ ]:
def get_rank(s):
  l = len(s)
  s1 = sorted(range(l), key=lambda k: s[k])
  s2 = [0] * l
  for idx, a in enumerate(s1):
    s2[a] = idx
  return s2


In [ ]:
get_rank([2, 3, 1, 4, 5, 3])

In [ ]:
df[emz_columns].apply(lambda x: get_rank(x), axis='columns')

In [ ]:
df[emz_columns]